In [1]:
# Searching a motif within the mic channel of a .kwd file
# the i-th motiff is in the file pattern_i.wav
# the raw data comes in file experiment.raw.kwd
# Definitions and functions
%matplotlib inline
import pdb
import sys
import pandas as pd
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib
import scipy.signal as sg
import math
import scipy as sp
import socket
import os
import wave
import struct
import h5py
from scipy.io import wavfile
from scipy.signal import spectrogram
import logging
from IPython.display import display
matplotlib.style.use('ggplot')
from ipywidgets import *
import yaml
from scipy.io import wavfile
from scipy.signal import resample
from __future__ import division

from helperephys.boner import stimprep
from swissknife.bci.core import expstruct as et
# Check wich computer to decide where the things are mounted
comp_name=socket.gethostname()

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.debug('all modules loaded')
logger.info('Computer: ' + comp_name)




/home/earneodo/.conda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-05-01 12:28:08,376 root         INFO     Computer: passaro


In [7]:
fold = os.path.join('/mnt/cube/earneodo', 'bci_zf', 'stim_data', 'b1159','001')
f_list = [f for f in os.listdir(fold) if f.split('.')[-1] == 'wav']

print(f_list)

['bos02.wav', 'bos03.wav', 'bos02-rev.wav', 'bos03-rev.wav', 'bos01.wav']


In [8]:

## Enter stim to tag manually
stims_to_tag = {'bos01': 1000,
                'bos03-rev': 7000,
                'bos02': 2000,
                'bos03': 3000,
                'bos04': 4000,
                'bos02-rev': 5000}


out_s_f = 44100

stimprep.create_bb_stim(f_list, fold, out_s_f, stim_tag_dict=stims_to_tag)
    

2018-05-01 18:12:55,875 helperephys.boner.stimprep INFO     Processing /mnt/cube/earneodo/bci_zf/stim_data/b1159/001/bos02.wav
2018-05-01 18:12:55,878 helperephys.boner.stimprep INFO     tag_freq = 2000
2018-05-01 18:12:56,092 helperephys.boner.stimprep INFO     Saved to /mnt/cube/earneodo/bci_zf/stim_data/b1159/001/bb_stim/bos02_tag.wav
2018-05-01 18:12:56,094 helperephys.boner.stimprep INFO     Processing /mnt/cube/earneodo/bci_zf/stim_data/b1159/001/bos03.wav
2018-05-01 18:12:56,096 helperephys.boner.stimprep INFO     tag_freq = 3000
2018-05-01 18:12:56,479 helperephys.boner.stimprep INFO     Saved to /mnt/cube/earneodo/bci_zf/stim_data/b1159/001/bb_stim/bos03_tag.wav
2018-05-01 18:12:56,481 helperephys.boner.stimprep INFO     Processing /mnt/cube/earneodo/bci_zf/stim_data/b1159/001/bos02-rev.wav
2018-05-01 18:12:56,483 helperephys.boner.stimprep INFO     tag_freq = 5000
2018-05-01 18:12:56,663 helperephys.boner.stimprep INFO     Saved to /mnt/cube/earneodo/bci_zf/stim_data/b1159/00

In [3]:
# THIS IS THE ACTUAL THING
def make_stereo_stim(wave_in, tag_freq, out_sf):
    in_sf, data = wavfile.read(wave_in)
    new_len = int(round(data.shape[0] * out_sf / in_sf))
    new_len = data.shape[0]
    #song_out = np.round(resample(data, new_len)).astype(np.int16)
    song_out = data
    tag_out = np.round(np.sin(2*np.pi*tag_freq*np.arange(new_len)/out_sf)*30000).astype(np.int16)
    
    return np.column_stack([song_out, tag_out])

bird = 'b1067'
stim_sess = '001'
out_sf = 30000

# stims_to_tag = {'bos': 1000,
#                 'con': 3000,
#                 'bos_rev': 5000}

# stims_to_tag = {'bos': 1000,
#                 'con': 2000,
#                 'bos_rev': 3000,
#                 'bos_syn': 4000}

stims_to_tag = {'bos01': 1000,
                'bos05-rev': 7000,
                'bos02': 2000,
                'bos03': 3000,
                'bos04': 4000,
                'bos05': 5000}

# stims_to_tag = {'bos05': 6000,
#                 'bos06': 500}

# stims_to_tag = {'bos01': 1000,
#                 'bos01_rev': 5000,
#                 'con01': 4000,
#                 'bos01_syn': 2000}

# stims_to_tag = {'bos01': 1000,
#                 'bos02': 2000,
#                 'bos03': 3000,
#                 'con01': 4000}


#stims_to_tag = {'bos01_rev': 500}
#stims_to_tag = {'bos03_rev': 1500}

#wave_out = os.path.join(os.path.abspath('/mnt/cube/earneodo/bci_zf/stim_data/z021/001'), 'bos_syn_tag.wav')
fn = et.file_names(bird, stim_sess)
for stim_name, tag_f in stims_to_tag.items():
    wave_in = os.path.join(fn['folders']['stim'], '{}.wav'.format(stim_name))
    print(wave_in)
    wave_out = os.path.join(fn['folders']['stim'], '{}_tag.wav'.format(stim_name))
    song_out = make_stereo_stim(wave_in, tag_f, out_sf)
    wavfile.write(wave_out, out_sf, song_out)
    print(wave_out)
    
tags_par_file = os.path.join(fn['folders']['stim'], 'stim_tags.yml')
with open(tags_par_file, 'w') as outfile:
    yaml.dump(stims_to_tag, outfile, default_flow_style=False)

/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos01.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos01_tag.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos05-rev.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos05-rev_tag.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos02.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos02_tag.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos03.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos03_tag.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos04.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos04_tag.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos05.wav
/mnt/cube/earneodo/bci_zf/stim_data/b1067/001/bos05_tag.wav


In [ ]:
tags_par_file = os.path.join(fn['folders']['stim'], 'stim_tags.yaml')
with open(tags_par_file, 'w') as outfile:
    yaml.dump(stims_to_tag, outfile, default_flow_style=False)

In [ ]:
bird = 'z023'
et.file_names(bird, '001')

In [ ]:
plt.plot(song_out.T[:1000])

In [ ]:
song_out.shape

In [ ]:
A1x16-5mm-50
passaro16-nn_acute